In [1]:
!pip install --upgrade transformers
!pip install pytorchvideo evaluate -q
!pip install --upgrade datasets
!pip install decord
!pip install -U huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB

In [2]:
from prediction import *

In [3]:
# load the model from disk
import pickle
filename = "finalized_model.sav"
clf = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
target_names = {0:'ang',
                1:'hap',
                2:'neu',
                3:'sad'}

In [5]:
text_mdl = "minoosh/finetuned_bert-base-on-IEMOCAP_2"
speech_mdl = "minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_2"
video_mdl = "minoosh/videomae-base-finetuned-IEMOCAP_2"

In [6]:
load_models(text_mdl, speech_mdl, video_mdl)

Some weights of the model checkpoint at minoosh/finetuned_bert-base-on-IEMOCAP_2 were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_2 were not used when initializing Wav2Vec2Model: ['classifier.weight', 'classifier.bias', 'projector.bias', 'projector.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


Some weights of the model checkpoint at minoosh/videomae-base-finetuned-IEMOCAP_2 were not used when initializing VideoMAEModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing VideoMAEModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoMAEModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
x = prepare_data('/content/Ses02F_impro04_M015.txt', '/content/Ses02F_impro04_M015.wav', '/content/Ses02F_impro04_M015.mp4')

  text_inputs = {k:torch.tensor(v).to(device) for k,v in text_inputs.items()}

  speech_input = {k:torch.tensor(v).to(device) for k,v in speech_input.items()}

  inputs = {k:torch.tensor(v).to(device) for k,v in inputs.items()}



In [19]:
y = clf.predict(x)

In [20]:
print(target_names[y[0]])

neu


In [12]:
import moviepy.editor as mp

# open video file
my_clip = mp.VideoFileClip(r"/content/WIN_20230624_12_04_41_Pro.mp4")
# extract pure audio file (no video)
my_clip.audio.write_audiofile(r"speech.wav")
# save pure video file (no audio)
new_clip = my_clip.without_audio()
new_clip.write_videofile("video.mp4")